In [1]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from civis.ml import ModelPipeline

import uuid
import json
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

In [2]:
# Default feature lists for Rainbow Modeling Frame (each number corresponds to number of features)
feature_table = civis.io.read_civis_sql(sql='select * from bernie_nmarchio2.feature_list order by sort_order asc', use_pandas = True, database='Bernie 2020')
feature_list_large = list(feature_table[(feature_table['frame_large'] == 1)]['feature_name']) + ['state_code']
feature_list_medium = list(feature_table[(feature_table['frame_medium'] == 1)]['feature_name']) + ['state_code']
feature_list_small = list(feature_table[(feature_table['frame_small'] == 1)]['feature_name']) + ['state_code']

additional_features = ['attempt_count', 'num_phones_1', 'num_phones_2', 'num_phones_3', 'num_phones_other']

feature_list_large.extend(additional_features)
feature_list_medium.extend(additional_features)
feature_list_small.extend(additional_features)

table_columns = civis.io.read_civis_sql(
    sql=f'''select ordinal_position as position, column_name, data_type 
    from information_schema.columns 
    where table_name = 'rainbow_modeling_frame' and table_schema = 'bernie_data_commons' and column_name != 'person_id'
    order by ordinal_position;''', use_pandas = True, database='Bernie DS')

#exclusion_list_466 = [e for e in list(table_columns['column_name']) if e not in feature_list_466] 

In [3]:
len(feature_list_small)

204

In [4]:
# USER INPUT CELL

# DV table parameters
DATABASE = 'Bernie DS'
# Primary key in both the DV table and the Modeling Frame
PRIMARY_KEY = 'person_id' 
# Table containing recoded Dependent Variables keyed to the PRIMARY_KEY
DV_TABLE = 'bernie_cherdeman.contactibility_outcomes_land'
# List of binarized dependent variables (accepts 1, 0, and null values) in DV_TABLE
DV_LIST = [ 'pickup_last'] # 'pickup_first', 'id_first',, 'id_last'


# Modeling frame table parameters
# Table containing covariates and keyed to PRIMARY_KEY
MODELING_FRAME = 'bernie_cherdeman.contactibility_modeling_frame_land'
# Columns in the Modeling Frame to exclude from feature list (i.e., strings or incomplete coverage)
EXCLUSION_COLUMNS = ['jsonid','state_code','census_block_group_2010', 'person_id', 'us_region', 
                     'pickup_first', 'pickup_last', 'pickup_ever', 'id_first', 'id_last', 'id_ever', 'is_cell_phone'
                    ,'civis_2020_race_native'
                    ,'civis_2020_race_black'
                    ,'civis_2020_race_latinx'
                    ,'civis_2020_race_asian'
                    ,'civis_2020_race_white'
                    ,'civis_2020_subeth_african_american'
                    ,'civis_2020_subeth_west_indian'
                    ,'civis_2020_subeth_haitian'
                    ,'civis_2020_subeth_african'
                    ,'civis_2020_subeth_other_black'
                    ,'civis_2020_subeth_mexican'
                    ,'civis_2020_subeth_cuban'
                    ,'civis_2020_subeth_puerto_rican'
                    ,'civis_2020_subeth_dominican'
                    ,'civis_2020_subeth_other_latin_american'
                    ,'civis_2020_subeth_other_hispanic'
                    ,'civis_2020_subeth_chinese'
                    ,'civis_2020_subeth_indian'
                    ,'civis_2020_subeth_filipino'
                    ,'civis_2020_subeth_japanese'
                    ,'civis_2020_subeth_vietnamese'
                    ,'civis_2020_subeth_korean'
                    ,'civis_2020_subeth_other_asian'
                    ,'civis_2020_subeth_hmong'
                    ,'civis_2020_cultural_religion_jewish'
                    ,'civis_2020_cultural_religion_mormon'
                    ,'civis_2020_cultural_religion_muslim'
                    ,'civis_2020_cultural_religion_catholic'
                    ,'civis_2020_cultural_religion_evangelical'
                    ,'civis_2020_cultural_religion_mainline_protestant'
                    ,'civis_2020_cultural_religion_hindu'
                    ,'civis_2020_cultural_religion_buddhist'
                    , 'religion_muslim'
                    ,'eth_white'
                    ,'eth_asian'
                    ,'eth_afam'
                    ,'eth_hispanic'
                    ,'eth_native'
                    ,'income_0k_25k'
                    ,'income_25k_50k'
                    ,'income_50k_100k'
                    ,'income_100k_150k'
                    ,'income_150k_plus']
# Output table parameters
# Schema to contain prediction tables
SCHEMA = 'bernie_cherdeman'
# String that will be concatenated in front of the output table's name
PREFIX = 'getthru'

# Sampling parameters
# Non-response training data
    # True: automatically select people not in DV_TABLE at random from Phoenix (assumes person_id is PRIMARY_KEY)
    # False: automatically select people where the DV equals 0 from the DV_TABLE
SAMPLE_FROM_PHOENIX = False
# Number of non-response classes per target class (default is 2) 
CLASS_BALANCE = 2
# Maximum number of targets to randomly sample from DV_TABLE
MAX_TARGET_COUNT = 40000

In [5]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())

In [6]:
# Counts of positive classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)


In [7]:
sql_count_targets

[['pickup_last'], ['68497']]

In [12]:
# Determing training table proportion of positives to negatives (to avoid class imbalance problems)
# sample_share = []
# for i in range(len(DV_LIST)):
#     if int(sql_count_targets[1][i]) > MAX_TARGET_COUNT:
#         sql_count_targets[1][i] = MAX_TARGET_COUNT
#     u = round(int(sql_count_targets[1][i])*CLASS_BALANCE)
#     sample_share.append(u)      

In [13]:
# for i in range(len(DV_LIST)):
#     dv_item = DV_LIST[i]
#     random_sample = sample_share[i]
#     if SAMPLE_FROM_PHOENIX is True:
#         zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
#     if SAMPLE_FROM_PHOENIX is False:
#         zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''


In [14]:
# Create training tables
#for i, dv_item in enumerate(DV_LIST):
#     if (int(sql_count_targets[1][i])*3) <= 1000:
    #feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
#     if (int(sql_count_targets[1][i])*3) > 1000 & (int(sql_count_targets[1][i])*3) <= 2000:
         
    #feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_medium])
#     if (int(sql_count_targets[1][i])*3) > 2000:
    #feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_large])
    #dv_item = DV_LIST[i]
#print(dv_item)
#     random_sample = sample_share[i]
#     if SAMPLE_FROM_PHOENIX is True:
#         zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
#     if SAMPLE_FROM_PHOENIX is False:
#         zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''
feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
dvs = "\n,".join(["{dv}".format(dv=d) for d in DV_LIST])

regions = ['South Atlantic',
            'New England',
            'Pacific',
            'West South Central',
            'East Midwest',
            'West Midwest',
            'Other',
            'East South Central',
            'Mountain']

for i, region in enumerate(regions):

    training_sql = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_land_training_{i} CASCADE;
                           CREATE TABLE {SCHEMA}.{PREFIX}_land_training_{i} DISTKEY({PRIMARY_KEY}) SORTKEY({PRIMARY_KEY}) AS (
                               select * 
                               from (select {PRIMARY_KEY}, {dvs} 
                                   from {DV_TABLE}
                                   where stratification < 5000
                                   and us_region = '{region}')
                               join (select {PRIMARY_KEY}, {feature_select}
                                     from {MODELING_FRAME}
                                     where us_region = '{region}')
                               using({PRIMARY_KEY})
                               );"""
    #print(training_sql)
    create_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
    create_training_sql.result().state
    print(f"{region} done")
    

South Atlantic done
New England done
Pacific done
West South Central done
East Midwest done
West Midwest done
Other done
East South Central done
Mountain done


In [15]:
#combine training sets into view
training_sql = f"""drop view if exists {SCHEMA}.{PREFIX}_training_land;
                   create view {SCHEMA}.{PREFIX}_training_land as (
                        (select * from {SCHEMA}.{PREFIX}_land_training_0)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_1)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_2)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_3)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_4)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_5)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_6)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_7)
                        union
                        (select * from {SCHEMA}.{PREFIX}_land_training_8)
                    );
                   """
combined_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
combined_training_sql.result().state

'succeeded'

In [16]:
# Train models
train_list = []
model_list = []

for i, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    for m in ['random_forest_classifier', 'sparse_logistic']:
    
        name = f"""{dv}_{m}_land_{datestamp}"""
        model = ModelPipeline(model=m,
                              dependent_variable=dv,
                              primary_key=PRIMARY_KEY,
                              excluded_columns=EXCLUSION_COLUMNS,
                              #calibration='sigmoid',
                              model_name=name,
                              memory_requested=15000#,
                              #disk_requested=5
                             )
    
        where_string = '{} is not null'.format(dv)
        # Use 
        train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training_land""", 
                            database_name=DATABASE,
                            sql_where=where_string#,
                            #fit_params={'sample_weight': WEIGHT_VAR}
                           )

        model_list.append(model)
        train_list.append(train) 

# name = f"""multioutcome_rf_{datestamp}"""
# model = ModelPipeline(model='random_forest_classifier',
#                       dependent_variable=DV_LIST,
#                       primary_key=PRIMARY_KEY,
#                       excluded_columns=EXCLUSION_COLUMNS,
#                       #calibration='sigmoid',
#                       model_name=name,
#                       memory_requested=15000#,
#                       #disk_requested=5
#                      )

# train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training""", 
#                     database_name=DATABASE,
#                     sql_where='''where pickup_first is not null 
#                                  and pickup_last is not null
#                                  and pickup_ever is not null
#                                  and id_first is not null
#                                  and id_last is not null
#                                  and id_ever is not null'''#,
#                     #fit_params={'sample_weight': WEIGHT_VAR}
#                    )

# model_list.append(model)
# train_list.append(train)


TRAINING >>> pickup_last


In [17]:
train_list[0]

<ModelFuture at 0x1120ef978 state=running>

In [33]:
# name = f"""multioutcome_rf_{datestamp}_v4"""
# model = ModelPipeline(model='random_forest_classifier',
#                       dependent_variable=DV_LIST,
#                       primary_key=PRIMARY_KEY,
#                       excluded_columns=EXCLUSION_COLUMNS,
#                       #calibration='sigmoid',
#                       model_name=name,
#                       memory_requested=15000#,
#                       #disk_requested=5
#                      )

# train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training""", 
#                     database_name=DATABASE,
#                     sql_where='''pickup_first is not null 
#                                  and pickup_last is not null
#                                  and pickup_ever is not null
#                                  and id_first is not null
#                                  and id_last is not null
#                                  and id_ever is not null'''#,
#                     #fit_params={'sample_weight': WEIGHT_VAR}
#                    )

# model_list.append(model)
# train_list.append(train)

In [18]:
len(train_list)

2

In [19]:
# Extract successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

    
model_output, train_output = zip(*((m, t) for m, t in zip(model_output, train_output) if t in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

Job success
Job success


In [20]:
print(len(jobs_list))
print(len(model_output))

2
2


In [21]:
# Generate validation metrics
metrics_list = []

for b in train_output:
    if b.job_id != 59761951:
        metric = {'job_id':b.job_id,
                  'run_id':b.run_id,
                  'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
                  'model': b.metadata['model']['model'],
                  'time_of_train_run': b.metadata['run']['time_of_run'],
                  'n_rows': b.metadata['data']['n_rows'],
                  'n_features': b.metadata['data']['n_cols'],
                  'auc': b.metadata['metrics']['roc_auc'],
                  'deciles': b.metadata['metrics']['deciles'],
                  'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
                  'accuracy': b.metadata['metrics']['accuracy'],
                  'p_correct': b.metadata['metrics']['p_correct'],
                  'pop_incidence_true': b.metadata['metrics']['pop_incidence_true'],
                  'feature_list':b.metadata['model']['parameters']['relvars']
                 }
        metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true','feature_list'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

,job_id,dv,model,time_of_train_run,n_rows,n_features,auc,deciles,confusion_matrix,accuracy,p_correct,pop_incidence_true,feature_list
run_id,,,,,,,,,,,,,
209330024,61465397,pickup_last,random_forest_classifier,2020-02-08T16:39:02Z,58477,160,0.733126,"[0.11935704514363885, 0.19018299982897213, 0.2...","[[26865, 5839], [13505, 12268]]",0.669203,"[0.8214591487279843, 0.4760020176153339]","[0.5592626160712759, 0.44073738392872414]","[civis_2020_marriage, civis_2020_children_pres..."
209330034,61465402,pickup_last,sparse_logistic,2020-02-08T16:38:10Z,58477,160,0.734928,"[0.12311901504787962, 0.20557550880793568, 0.2...","[[24908, 7796], [11157, 14616]]",0.675890,"[0.761619373776908, 0.567105109998836]","[0.5592626160712759, 0.44073738392872414]","[civis_2020_children_present, civis_2020_ideol..."


In [22]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.{PREFIX}_validation_land_{datestamp}', 
                                                 existing_table_rows='drop')


In [10]:
# Score the voterfile
scores_list = []
#for m,t in zip(model_output, train_output):
m = civis.ml.ModelPipeline.from_existing(61465397) 
DV_NAME = 'pickup_last'#''.join(t.metadata['run']['configuration']['data']['y'])
MODEL = 'random_forest_classifier'
#     DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
#     MODEL = t.metadata['model']['model']
print(DV_NAME, MODEL)
SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{MODEL}_{DV_NAME}_land_nodemo_{datestamp}'
scores_list.append(SCORES_TABLE)
scores = m.predict(primary_key=PRIMARY_KEY,
                   database_name=DATABASE, 
                   table_name='bernie_cherdeman.contactibility_modeling_frame_land_co_tx',
                   if_exists='drop',
                   output_table=SCORES_TABLE,
                   disk_space=10)
scores.result()



pickup_last random_forest_classifier


ModelError: Model run failed with stack trace:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/civismodel-2.2.4-py3.6.egg/civismodel/predict/core.py", line 320, in spawn_predict_jobs
    output = parallel(delayed(func)(**args) for args in iterkwargs)
  File "/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 789, in __call__
    self.retrieve()
  File "/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 701, in retrieve
    self._output.extend(job.get())
  File "/opt/conda/lib/python3.6/site-packages/civis/parallel.py", line 728, in get
    raise self.result
joblib.my_exceptions.TransportableException: TransportableException
___________________________________________________________________________
EmptyResultError                                   Mon Feb 10 21:44:16 2020
PID: 1                                  Python 3.6.4: /opt/conda/bin/python
..........................................

In [ ]:
# Generate SQL for final output table and drop intermediary tables
input_train_list = []
output_score_list = []
for i in range(len(DV_LIST)):
    input_train = f"{SCHEMA}.{PREFIX}_training_{i}"
    input_train_list.append(input_train)
    output_score = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    output_score_list.append(output_score)

drop_input_train_sql = "\n".join(["drop table if exists {tbl};".format(tbl=v) for v in input_train_list])
drop_output_score_sql = "\n".join(["drop table if exists {tbl};".format(tbl=t) for t in output_score_list])  
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(output_score_list) > 1:
    for i in output_score_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [ ]:
output_table_sql = f"""
set query_group to 'importers';
set wlm_query_slot_count to 3;
DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(output_score_list[0]) + ''.join(join_table) +');'  


In [ ]:
print(output_table_sql)

In [ ]:
# Create final output table
create_output_table = civis.io.query_civis(sql=output_table_sql, database=DATABASE)
create_output_table.result().state


In [ ]:
# Drop intermediary tables
drop_input_train_query = civis.io.query_civis(sql=drop_input_train_sql, database=DATABASE)
drop_input_train_query.result().state

drop_output_score_query = civis.io.query_civis(sql=drop_output_score_sql, database=DATABASE)
drop_output_score_query.result().state

In [ ]:
print(drop_input_train_sql)
print(drop_output_score_sql)

In [ ]:
# Grant team on tables
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_output_{datestamp} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_validation_{datestamp} TO GROUP bernie_data;
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

In [ ]:
print(grant_statement)